In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import tensorflow as tf
from tensorflow import keras

2023-12-19 16:14:36.534610: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-19 16:14:36.536901: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 16:14:36.582289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 16:14:36.583023: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 16:14:37.298767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## data generator:

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import xception

In [3]:
data_gen = ImageDataGenerator(preprocessing_function=xception.preprocess_input)

train_ds = data_gen.flow_from_directory(
    './Fast Food Classification V2/Train',
    target_size=(100, 100),
    batch_size=32
)

val_ds = data_gen.flow_from_directory(
    './Fast Food Classification V2/Valid',
    target_size=(100, 100),
    batch_size=32
)

Found 15000 images belonging to 10 classes.
Found 3500 images belonging to 10 classes.


## Model:

In [4]:
base_model = xception.Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(100, 100, 3)
)

base_model.trainable = False

2023-12-19 09:07:10.870696: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-19 09:07:10.905404: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


83683744/83683744 [==============================] - 1s 0us/step


In [5]:
inputs = keras.Input(shape=(100, 100, 3))
base = base_model(inputs, training=False)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = keras.layers.Dense(10)(vectors)
model = keras.Model(inputs, outputs)

In [6]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


469/469 [==============================] - 330s 699ms/step - loss: 2.2339 - accuracy: 0.5501 - val_loss: 2.4473 - val_accuracy: 0.5551
Epoch 2/10
469/469 [==============================] - 329s 702ms/step - loss: 1.7174 - accuracy: 0.6447 - val_loss: 2.6295 - val_accuracy: 0.5637
Epoch 3/10
469/469 [==============================] - 331s 708ms/step - loss: 1.6808 - accuracy: 0.6710 - val_loss: 2.7399 - val_accuracy: 0.5683
Epoch 4/10
469/469 [==============================] - 327s 697ms/step - loss: 1.6378 - accuracy: 0.6912 - val_loss: 3.3409 - val_accuracy: 0.5463
Epoch 5/10
469/469 [==============================] - 339s 725ms/step - loss: 1.4511 - accuracy: 0.7151 - val_loss: 3.4345 - val_accuracy: 0.5583
Epoch 6/10
469/469 [==============================] - 406s 867ms/step - loss: 1.4685 - accuracy: 0.7205 - val_loss: 3.5982 - val_accuracy: 0.5549
Epoch 7/10
469/469 [==============================] - 327s 697ms/step - loss: 1.4326 - accuracy: 0.7280 - val_loss: 3.9481 - val_accura

In [ ]:
keras.models.save_model(model, 'v1-xception.keras')

---

In [4]:
modified_xception_model = keras.models.load_model('v1-xception.keras')
# keras.models.save_model(modified_xception_model, 'v1-xception.keras')

2023-12-19 16:15:12.662114: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-19 16:15:12.695274: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
modified_xception_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 20881970 (79.66 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 20861480 (79.58 MB)
_________________________________________________________________


In [6]:
modified_xception_model.predict(train_ds[0][0])

1/1 [==============================] - 1s 1s/step


array([[  4.7928786 , -11.672298  ,   6.203593  , -11.354408  ,
          7.7251086 ,  -7.755991  ,  -0.99260664,  -5.86283   ,
          2.0686114 ,   0.9055846 ],
       [-11.842133  ,  10.197781  , -22.110275  , -17.755285  ,
          0.6128199 ,  25.429972  , -11.722082  ,   0.69671917,
         -0.37566036,   4.7126384 ],
       [ -1.4696295 ,  12.517413  ,  -7.6251607 ,  -4.2995467 ,
        -12.292243  ,   6.701681  ,  -1.3782383 ,  18.097145  ,
         -4.2126985 ,  -7.7437344 ],
       [ -3.8187077 ,  -6.180215  ,  -7.4111614 , -11.581553  ,
        -17.797405  ,  19.060682  ,  -4.219546  ,  -3.7384565 ,
         10.752395  ,   2.5731332 ],
       [ 11.155827  , -11.297539  ,  -6.9895797 ,  -6.4904356 ,
          0.13787466,   4.4801903 ,  -3.247439  ,  -1.6726093 ,
         -2.1309898 ,   2.9171565 ],
       [ 18.59045   ,  -2.782077  ,   3.9185948 ,  -0.5755358 ,
         -9.851805  ,   3.3013752 , -19.617939  ,  -2.6413572 ,
          7.420755  ,   1.1178675 ],
       [ -

In [ ]:
#plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.xticks(np.arange(10))
plt.legend()